In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import itertools
from tokenizers.models import WordLevel
from tokenizers import Tokenizer, models, pre_tokenizers, trainers
from transformers import LineByLineTextDataset
from joblib import Parallel, delayed

from transformers import RobertaTokenizerFast
from transformers import LineByLineTextDataset
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def log_return(series):
    return np.log(series).diff()

def book_preprocessor(file_path, delta):
    df = pd.read_parquet(file_path)[['time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_size1', 'ask_size1']]
    df['wap1'] = calc_wap1(df)
    df['i'] = ((df['wap1'] - wap1_min)/delta).astype(int)
    return list(df.groupby(['time_id']).apply(convert_to_str).values)
    
def convert_to_str(g):
    s = ''.join(str(g.i.values)).replace('\n', '')
    return s[1: len(s) - 1] + ' . '

def preprocessor(list_stock_ids, delta, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        return book_preprocessor(file_path_book, delta)
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    return df

In [ ]:
delta = 0.0002
wap1_min = 0.8830628395080566
wap1_max = 1.1270768642425537
vocab_size = int((wap1_max - wap1_min)/delta) + 2
num_secs_max = 600

In [ ]:
data_dir = '../input/optiver-realized-volatility-prediction/'

idx0 = 0
idx1 = 2
train, test = read_train_test()
train_stock_ids = train['stock_id'].unique()[idx0:idx1]
train_ = preprocessor(train_stock_ids, delta, is_train = True)
merged_data = list(itertools.chain(*train_))

textfile = open("train_data.txt", "w")
for element in merged_data:
    textfile.write(element +'\n')
textfile.close()

del train_
del merged_data

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('../input/krv-tokenizer')
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=".//train_data.txt",
    block_size=128,
)

In [ ]:
from_pretrained = True

config = RobertaConfig(
    vocab_size=len(tokenizer.get_vocab()),
    max_position_embeddings=600,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

if from_pretrained:
    model = RobertaForMaskedLM.from_pretrained('../input/bert-for-optiver')
else:
    model = RobertaForMaskedLM(config=config)

print(f'Num of model parameters = {model.num_parameters()}')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./optiver",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

In [ ]:
model.save_pretrained('model/krv_model')

In [ ]:
from IPython.display import FileLinks
FileLinks(r'model')